[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rbernalc/pytorch-mx/blob/main/tutorials/basic/Tutorials_Basic_colab.ipynb)

### Tutoriales Basicos  PyTorch

# Tensores

Los tensores son una estructura de datos especializada similar a los arreglos y matrices.
En PyTorch, usamos tensores para codificar las entradas y salidas de un modelo, así como los parámetros del modelo.

Los tensores son similares a los ndarrays de [NumPy](https://numpy.org/), excepto que los tensores pueden ejecutarse en GPUs u otros aceleradores de hardware. De hecho, los tensores y los arreglos de NumPy a menudo pueden compartir la misma memoria subyacente, eliminando la necesidad de copiar datos. Los tensores también están optimizados para diferenciación automática (veremos más sobre eso más adelante en la sección [Autograd](autogradqs_tutorial.html)). Si estás familiarizado con ndarrays, te sentirás como en casa con la API de Tensores. ¡Si no, sigue adelante!

In [ ]:
import torch
import numpy as np

## Inicializar un Tensor

Los tensores pueden ser inicializados de varias maneras. Echa un vistazo a los siguientes ejemplos:

### Directamente desde datos

Los tensores pueden ser creados directamente desde datos. El tipo de dato se infiere automáticamente.

In [ ]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

x_data.type()

### Desde un arreglo NumPy

Los tensores pueden ser creados desde arreglos NumPy (y viceversa).

In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
x_np

### Desde otro tensor:

El nuevo tensor mantiene las propiedades (forma, tipo de dato) del tensor argumento, a menos que se anule explícitamente.

In [ ]:
x_ones = torch.ones_like(x_data) # mantiene las propiedades de x_data
print(f"Tensor de Unos: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # anula el tipo de dato de x_data
print(f"Tensor Aleatorio: \n {x_rand} \n")

### Con valores aleatorios o constantes:

Para el siguiente ejemplo ``shape`` es una tupla de dimensiones del tensor. En las funciones de abajo, determina la dimensionalidad del tensor de salida.

In [ ]:
shape = (2,3)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Tensor Aleatorio: \n {rand_tensor} \n")
print(f"Tensor de Unos: \n {ones_tensor} \n")
print(f"Tensor de Ceros: \n {zeros_tensor}")

---

## Atributos de un Tensor

Los atributos del tensor describen su forma, tipo de dato y el dispositivo en el que están almacenados.

In [ ]:
tensor = torch.rand(3,4)

print(f"Forma del tensor: {tensor.shape}")
print(f"Tipo de dato del tensor: {tensor.dtype}")
print(f"Dispositivo donde está almacenado el tensor: {tensor.device}")

---

## Operaciones en Tensores

PyTorch contiene más de 1200 operaciones de tensores, incluyendo aritmética, álgebra lineal, manipulación de matrices (transponer,
indexar, rebanar), muestreo y más, las cuales están descritas exhaustivamente [aquí](https://pytorch.org/docs/stable/torch.html).

Cada una de estas operaciones puede ejecutarse en la CPU y en [Acelerador](https://pytorch.org/docs/stable/torch.html#accelerators)
como CUDA, MPS, MTIA, o XPU. Al usar Colab, asigna un acelerador yendo a Runtime > Change runtime type > GPU.

Por defecto, los tensores se crean en la CPU. Necesitamos mover explícitamente los tensores al acelerador usando
el método ``.to`` (después de verificar la disponibilidad del acelerador). Ten en cuenta que copiar tensores grandes
entre dispositivos puede ser costoso en términos de tiempo y memoria!

In [ ]:
# Movemos nuestro tensor al acelerador actual si está disponible
if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())

A continuación se muestran algunas de las operaciones de la lista.
Si estás familiarizado con la API de NumPy, encontrarás que la API de Tensores es muy fácil de usar.

### Indexación y rebanado (slicing) estándar tipo numpy:

In [ ]:
tensor = torch.ones(4, 4)
print(f"Primera fila: {tensor[0]}")
print(f"Primera columna: {tensor[:, 0]}")
print(f"Última columna: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

### Unir tensores

Puedes usar ``torch.cat`` para concatenar una secuencia de tensores a lo largo de una dimensión dada.
Existe otro oeprador de union llamadao [torch.stack](https://pytorch.org/docs/stable/generated/torch.stack.html), que es sutilmente diferente de ``torch.cat``.

In [ ]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

### Operaciones aritméticas

In [ ]:
# Esto calcula la multiplicación de matrices entre dos tensores usando matmul o su función equivalente @. 
# y1, y2, y3 tendrán el mismo valor
# ``tensor.T`` devuelve la transpuesta de un tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)

In [ ]:
# Esto calcula el producto elemento por elemento. z1, z2, z3 tendrán el mismo valor
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

### Tensores de un solo elemento

Si tienes un tensor de un elemento, por ejemplo despues de sumar todos los valores de un mismo tensor, puedes convertirlo a un valor numérico de Python usando ``item()``:

In [ ]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

### Operaciones in-place

Las operaciones que almacenan el resultado en el mismo lugar (operando) se llaman in-place. Se denotan con un sufijo ``_``.
Por ejemplo: ``x.copy_(y)``, ``x.t_()``, cambiarán el contenido de ``x``.

In [ ]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

**Nota:**
Las operaciones in-place ahorran algo de memoria, pero pueden ser problemáticas al calcular derivadas debido a una pérdida inmediata
del historial. Por lo tanto, se desaconseja su uso.

---

## Puente con NumPy

Los tensores en la CPU y los arreglos NumPy pueden compartir sus ubicaciones de memoria subyacente,  de manera que cambiar los valores de uno cambiará los valores del otro.

### Tensor a arreglo NumPy

In [ ]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

Un cambio en el tensor se refleja en el arreglo NumPy.

In [ ]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

### Arreglo NumPy a Tensor

In [ ]:
n = np.ones(5)
t = torch.from_numpy(n)

Los cambios en el arreglo NumPy se reflejan en el tensor.

In [ ]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

# Diferenciación Automática con ``torch.autograd``

Al entrenar redes neuronales, el algoritmo más frecuentemente usado es
**back propagation**. En este algoritmo, los parámetros (pesos del modelo) se
ajustan de acuerdo al **gradiente** de la función de pérdida con respecto al
parámetro dado.

Para calcular esos gradientes, PyTorch tiene un motor de diferenciación automática
integrado llamado ``torch.autograd``. Soporta el cálculo automático del gradiente para cualquier
gráfico computacional.

Considera la red neuronal más simple de una capa, con entrada ``x``, parámetros ``w`` y ``b``, 
y alguna función de pérdida. Puede ser definida en PyTorch de la siguiente manera:

In [ ]:
import torch

x = torch.ones(5)  # tensor de entrada
y = torch.zeros(3)  # salida esperada
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [ ]:
print(z)
print(y)

In [ ]:
print(loss)

En esta red, ``w`` y ``b`` son **parámetros**, los cuales necesitamos
optimizar. Por lo tanto, necesitamos poder calcular los gradientes de la
función de pérdida con respecto a esas variables. Para hacer eso, establecemos
la propiedad ``requires_grad`` de esos tensores.

> Puedes establecer el valor de ``requires_grad`` al crear un tensor, o después usando el método ``x.requires_grad_(True)``.

Una función que aplicamos a los tensores para construir el gráfico computacional
es de hecho un objeto de la clase ``Function``. Este objeto sabe cómo calcular la función
en la dirección *hacia adelante*, y también cómo calcular su derivada durante el paso
de *propagación hacia atrás*. Una referencia a la función de propagación hacia atrás se
almacena en la propiedad ``grad_fn`` de un tensor. Puedes encontrar más información sobre ``Function`` en la [documentación](https://pytorch.org/docs/stable/autograd.html#function).

Para optimizar los pesos de los parámetros en la red neuronal, necesitamos
calcular las derivadas de nuestra función de pérdida con respecto a los parámetros,
es decir, necesitamos $\frac{\partial loss}{\partial w}$ y $\frac{\partial loss}{\partial b}$
bajo algunos valores fijos de ``x`` y ``y``. Para calcular esas derivadas, llamamos
``loss.backward()``, y luego recuperamos los valores de ``w.grad`` y ``b.grad``.

In [ ]:
# Esto calcula la multiplicación de matrices entre dos tensores usando matmul o su función equivalente @. 
# y1, y2, y3 tendrán el mismo valor
# ``tensor.T`` devuelve la transpuesta de un tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)

In [ ]:
# Esto calcula el producto elemento por elemento. z1, z2, z3 tendrán el mismo valor
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

### Tensores de un solo elemento

Si tienes un tensor de un elemento, por ejemplo despues de sumar todos los valores de un mismo tensor, puedes convertirlo a un valor numérico de Python usando ``item()``:

In [ ]:
agg = tensor.sum()
agg_item = agg.item()
print(agg_item, type(agg_item))

### Operaciones in-place

Las operaciones que almacenan el resultado en el mismo lugar (operando) se llaman in-place. Se denotan con un sufijo ``_``.
Por ejemplo: ``x.copy_(y)``, ``x.t_()``, cambiarán el contenido de ``x``.

In [ ]:
print(f"{tensor} \n")
tensor.add_(5)
print(tensor)

**Nota:**
Las operaciones in-place ahorran algo de memoria, pero pueden ser problemáticas al calcular derivadas debido a una pérdida inmediata
del historial. Por lo tanto, se desaconseja su uso.

---

## Puente con NumPy

Los tensores en la CPU y los arreglos NumPy pueden compartir sus ubicaciones de memoria subyacente,  de manera que cambiar los valores de uno cambiará los valores del otro.

### Tensor a arreglo NumPy

In [ ]:
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n: {n}")

Un cambio en el tensor se refleja en el arreglo NumPy.

In [ ]:
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

### Arreglo NumPy a Tensor

In [ ]:
n = np.ones(5)
t = torch.from_numpy(n)

Los cambios en el arreglo NumPy se reflejan en el tensor.

In [ ]:
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

# Diferenciación Automática con ``torch.autograd``

Al entrenar redes neuronales, el algoritmo más frecuentemente usado es
**back propagation**. En este algoritmo, los parámetros (pesos del modelo) se
ajustan de acuerdo al **gradiente** de la función de pérdida con respecto al
parámetro dado.

Para calcular esos gradientes, PyTorch tiene un motor de diferenciación automática
integrado llamado ``torch.autograd``. Soporta el cálculo automático del gradiente para cualquier
gráfico computacional.

Considera la red neuronal más simple de una capa, con entrada ``x``, parámetros ``w`` y ``b``, 
y alguna función de pérdida. Puede ser definida en PyTorch de la siguiente manera:

In [ ]:
import torch

x = torch.ones(5)  # tensor de entrada
y = torch.zeros(3)  # salida esperada
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [ ]:
print(z)
print(y)

In [ ]:
print(loss)

En esta red, ``w`` y ``b`` son **parámetros**, los cuales necesitamos
optimizar. Por lo tanto, necesitamos poder calcular los gradientes de la
función de pérdida con respecto a esas variables. Para hacer eso, establecemos
la propiedad ``requires_grad`` de esos tensores.

> Puedes establecer el valor de ``requires_grad`` al crear un tensor, o después usando el método ``x.requires_grad_(True)``.

Una función que aplicamos a los tensores para construir el gráfico computacional
es de hecho un objeto de la clase ``Function``. Este objeto sabe cómo calcular la función
en la dirección *hacia adelante*, y también cómo calcular su derivada durante el paso
de *propagación hacia atrás*. Una referencia a la función de propagación hacia atrás se
almacena en la propiedad ``grad_fn`` de un tensor. Puedes encontrar más información sobre ``Function`` en la [documentación](https://pytorch.org/docs/stable/autograd.html#function).

Para optimizar los pesos de los parámetros en la red neuronal, necesitamos
calcular las derivadas de nuestra función de pérdida con respecto a los parámetros,
es decir, necesitamos $\frac{\partial loss}{\partial w}$ y $\frac{\partial loss}{\partial b}$
bajo algunos valores fijos de ``x`` y ``y``. Para calcular esas derivadas, llamamos
``loss.backward()``, y luego recuperamos los valores de ``w.grad`` y ``b.grad``.

In [ ]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

> **Nota**
> - Solo podemos obtener las propiedades ``grad`` para los nodos hoja del gráfico computacional, que tienen la propiedad ``requires_grad`` establecida en ``True``. Para todos los otros nodos en nuestro gráfico, los gradientes no estarán disponibles.
> - Solo podemos realizar cálculos de gradiente usando ``backward`` una vez en un gráfico dado por razones de rendimiento. Si necesitamos hacer varias llamadas ``backward`` en el mismo gráfico, necesitamos pasar ``retain_graph=True`` a la llamada ``backward``.

In [ ]:
loss.backward()
print(w.grad)
print(b.grad)


## Deshabilitando el Seguimiento del Gradiente

Por defecto, todos los tensores con ``requires_grad=True`` están siguiendo su
historial computacional y soportan el cálculo del gradiente. Sin embargo, hay algunos casos donde no necesitamos
hacer eso, por ejemplo, cuando hemos entrenado el modelo y solo queremos aplicarlo a
algunos datos de entrada, es decir, solo queremos hacer cálculos *hacia adelante* a través de la red.
Podemos detener el rastreo de los cálculos envolviendo nuestro código de cálculo con el bloque ``torch.no_grad()``:

Otra forma de lograr el mismo resultado es usando el método ``detach()`` en el tensor:

El tensor resultante no tiene ``requires_grad=True``

Hay razones por las que podrías querer deshabilitar el seguimiento del gradiente:
  - Para marcar algunos parámetros en tu red neuronal como **parámetros congelados**.
  - Para **acelerar los cálculos** cuando solo estás haciendo un paso hacia adelante, porque los cálculos en tensores que no rastrean gradientes serían más eficientes.

In [ ]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

Otra forma de lograr el mismo resultado es usando el método ``detach()`` en el tensor:

In [ ]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

---

## Más sobre el Gráfico Computacional

Conceptualmente, autograd mantiene un registro de datos (tensores) y todas las operaciones ejecutadas
(junto con los nuevos tensores resultantes) en un gráfico acíclico dirigido (DAG) que consiste de
objetos [Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function). En este DAG, las hojas
son los tensores de entrada, las raíces son los tensores de salida. Rastreando este gráfico desde las raíces hasta las hojas, 
puedes calcular automáticamente los gradientes usando la regla de la cadena.

En un paso hacia adelante, autograd hace dos cosas simultáneamente:

- ejecuta la operación solicitada para calcular un tensor resultante
- mantiene la *función de gradiente* de la operación en el DAG.

El paso hacia atrás se inicia cuando se llama ``.backward()`` en la raíz del DAG.
``autograd`` luego:

- calcula los gradientes de cada ``.grad_fn``,
- los acumula en el atributo ``.grad`` del tensor respectivo
- usando la regla de la cadena, se propaga hasta los tensores hoja.

> **Nota**
>
> **DAGs son dinámicos en PyTorch**
> Una cosa importante a notar es que el gráfico se recrea desde cero; después de cada llamada a ``.backward()``, autograd comienza a poblar un nuevo gráfico. Esto es exactamente lo que te permite usar declaraciones de flujo de control en tu modelo; puedes cambiar la forma, tamaño y operaciones en cada iteración si es necesario.

## Gradientes de Tensores y Funciones Jacobianas

En muchos casos, tenemos una función escalar de pérdida, y necesitamos calcular el gradiente 
con respecto a algunos parámetros. Sin embargo, hay casos donde la función de salida
es un tensor arbitrario. En este caso, PyTorch te permite calcular el llamado **producto Jacobiano-vector**, en lugar de la matriz Jacobiana actual.

Para un vector función $\vec{y}=f(\vec{x})$, donde $\vec{x}=\langle x_1,\ldots,x_n\rangle$ y
$\vec{y}=\langle y_1,\ldots,y_m\rangle$, un gradiente de $\vec{y}$ con respecto a
$\vec{x}$ es dado por la **matriz Jacobiana**:

$$J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)$$

En lugar de calcular la matriz Jacobiana en sí misma, PyTorch te permite calcular el **producto Jacobiano-vector** $J^T \cdot v$ para un vector dado $v$. Esto se logra
llamando a ``backward`` con $v$ como argumento. El tamaño de $v$ debería ser el mismo que
el tamaño del tensor original, con respecto al cual queremos calcular el producto:


In [ ]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"Primer llamado\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSegundo llamado\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nLlamado después de poner los gradientes en cero\n{inp.grad}")

Observa que cuando llamamos ``backward`` por segunda vez con los mismos argumentos, el valor de
los gradientes es diferente. Esto sucede porque al hacer la propagación hacia atrás, PyTorch
**acumula los gradientes**, es decir, el valor de los gradientes calculados se suma al atributo ``.grad`` 
de todos los nodos hoja del gráfico computacional. Si quieres calcular los gradientes apropiados, necesitas
poner la propiedad ``.grad`` en cero antes. En el entrenamiento de la vida real esto es hecho para nosotros por el optimizador.

> **Nota**
>
> Anteriormente estábamos llamando a la función ``backward()`` sin parámetros. Esto es esencialmente equivalente a 
> llamar ``backward(torch.tensor(1.0))``, lo cual es una forma útil de calcular los gradientes en el caso de 
> una función escalar, como la pérdida durante el entrenamiento de redes neuronales.

---

## Lectura Adicional

- [API de Autograd](https://pytorch.org/docs/stable/autograd.html)

# Conjuntos de Datos y DataLoaders

El código para procesar muestras de datos puede volverse desordenado y difícil de mantener; idealmente queremos que nuestro código de conjunto de datos
esté desacoplado de nuestro código de entrenamiento del modelo para mejor legibilidad y modularidad.
PyTorch proporciona dos primitivas de datos: ``torch.utils.data.DataLoader`` y ``torch.utils.data.Dataset``
que te permiten usar conjuntos de datos pre-cargados así como tus propios datos.
``Dataset`` almacena las muestras y sus etiquetas correspondientes, y ``DataLoader`` envuelve un iterable alrededor
del ``Dataset`` para habilitar acceso fácil a las muestras.

Las bibliotecas de dominio de PyTorch proporcionan un número de conjuntos de datos pre-cargados (como FashionMNIST) que
son subclases de ``torch.utils.data.Dataset`` e implementan funciones específicas para los datos particulares.
Pueden usarse para prototipar y hacer benchmark de tu modelo. Puedes encontrarlos
aquí: [Conjuntos de Datos de Imágenes](https://pytorch.org/vision/stable/datasets.html),
[Conjuntos de Datos de Texto](https://pytorch.org/text/stable/datasets.html), y
[Conjuntos de Datos de Audio](https://pytorch.org/audio/stable/datasets.html)

## Cargar un Conjunto de Datos

Aquí hay un ejemplo de cómo cargar el conjunto de datos [Fashion-MNIST](https://research.zalando.com/project/fashion_mnist/fashion_mnist/) de TorchVision.
Fashion-MNIST es un conjunto de datos de imágenes de artículos de Zalando que consiste en 60,000 ejemplos de entrenamiento y 10,000 ejemplos de prueba.
Cada ejemplo comprende una imagen en escala de grises de 28×28 y una etiqueta asociada de una de 10 clases.

Cargamos el [Conjunto de Datos FashionMNIST](https://pytorch.org/vision/stable/datasets.html#fashion-mnist) con los siguientes parámetros:
- ``root`` es la ruta donde se almacenan los datos de entrenamiento/prueba,
- ``train`` especifica conjunto de datos de entrenamiento o prueba,
- ``download=True`` descarga los datos de internet si no están disponibles en ``root``.
- ``transform`` y ``target_transform`` especifican las transformaciones de características y etiquetas

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

## Iterar y Visualizar el Conjunto de Datos

Podemos indexar ``Datasets`` manualmente como una lista: ``training_data[index]``.
Usamos ``matplotlib`` para visualizar algunas muestras en nuestros datos de entrenamiento.

In [ ]:
labels_map = {
    0: "Camiseta",
    1: "Pantalón", 
    2: "Suéter",
    3: "Vestido",
    4: "Abrigo",
    5: "Sandalia",
    6: "Camisa",
    7: "Zapatilla",
    8: "Bolso",
    9: "Botín",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

## Crear un Conjunto de Datos Personalizado para tus archivos

Una clase Dataset personalizada debe implementar tres funciones: `__init__`, `__len__`, y `__getitem__`.
Echa un vistazo a esta implementación; las imágenes de FashionMNIST están almacenadas
en un directorio ``img_dir``, y sus etiquetas están almacenadas separadamente en un archivo CSV ``annotations_file``.

En las siguientes secciones, desglosaremos qué está sucediendo en cada una de estas funciones.

In [ ]:
import os
import pandas as pd
from torchvision.io import decode_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = decode_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

### `__init__`

La función __init__ se ejecuta una vez al instanciar el objeto Dataset. Inicializamos
el directorio que contiene las imágenes, el archivo de anotaciones, y ambas transformaciones (cubiertas
en más detalle en la siguiente sección).

El archivo labels.csv se ve así:

```
tshirt1.jpg, 0
tshirt2.jpg, 0
......
ankleboot999.jpg, 9
```

In [ ]:
def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
    self.img_labels = pd.read_csv(annotations_file)
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform

### `__len__`

La función __len__ devuelve el número de muestras en nuestro conjunto de datos.

Ejemplo:

In [ ]:
def __len__(self):
    return len(self.img_labels)

### `__getitem__`

La función __getitem__ carga y devuelve una muestra del conjunto de datos en el índice dado ``idx``.
Basado en el índice, identifica la ubicación de la imagen en el disco, la convierte a un tensor usando ``read_image``, 
recupera la etiqueta correspondiente desde el archivo csv en ``self.img_labels``, llama a las funciones de transformación 
en ellos (si aplica), y devuelve la muestra del tensor y la etiqueta correspondiente en una tupla.

In [ ]:
def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = decode_image(img_path)
    label = self.img_labels.iloc[idx, 1]
    if self.transform:
        image = self.transform(image)
    if self.target_transform:
        label = self.target_transform(label)
    return image, label

---

## Preparar tus datos para el entrenamiento con DataLoaders

El ``Dataset`` recupera las características de nuestro conjunto de datos y etiqueta una muestra a la vez. Mientras entrenamos un modelo, típicamente queremos pasar muestras en "minibatches", reordenar los datos en cada época para reducir el overfitting del modelo, y usar el multiprocesamiento de Python para acelerar la recuperación de datos.

``DataLoader`` es un iterable que abstrae esta complejidad para nosotros en una API fácil.

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

## Iterar a través del DataLoader

Hemos cargado ese conjunto de datos en el ``DataLoader`` y podemos iterar a través del conjunto de datos según sea necesario.
Cada iteración a continuación devuelve un lote de ``train_features`` y ``train_labels`` (conteniendo elementos de ``batch_size=64`` cada uno).
Debido a que especificamos ``shuffle=True``, después de que iteremos sobre todos los lotes, los datos se mezclan (para un control más detallado sobre el orden de carga de los datos, echa un vistazo a [Samplers](https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler).

In [ ]:
# Mostrar imagen y etiqueta.
train_features, train_labels = next(iter(train_dataloader))
print(f"Forma del lote de características: {train_features.size()}")
print(f"Forma del lote de etiquetas: {train_labels.size()}")
print(train_features[0].shape)
img = train_features[0].squeeze()
print(img.shape)
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Etiqueta: {label}")

---

## Lectura Adicional
- [torch.utils.data API](https://pytorch.org/docs/stable/data.html)

# Transformaciones

Los datos no siempre vienen en su forma final procesada que es requerida para
entrenar algoritmos de aprendizaje automático. Usamos **transformaciones** para realizar
alguna manipulación de los datos y hacerlos adecuados para el entrenamiento.

Todas las datasets de TorchVision tienen dos parámetros - ``transform`` para modificar
las características e ``target_transform`` para modificar las etiquetas - que
aceptan callables conteniendo la lógica de transformación.
El módulo [torchvision.transforms](https://pytorch.org/vision/stable/transforms.html) ofrece
varias transformaciones predefinidas comúnmente usadas listas para usar.

Las características de FashionMNIST están en formato de imagen PIL, y las etiquetas son enteros.
Para el entrenamiento, necesitamos las características como tensores normalizados, y las etiquetas
como tensores de one-hot encoded. Para hacer estas transformaciones, usamos ``ToTensor`` e ``Lambda``.

In [ ]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

## ToTensor()

[ToTensor](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.ToTensor)
convierte una imagen PIL o arreglo NumPy a un ``FloatTensor``. y escala
los valores de intensidad de pixel de la imagen en el rango [0., 1.]

## Lambda Transforms

Las transformaciones Lambda aplican cualquier función lambda definida por el usuario. Aquí, definimos una función
para convertir el entero a un tensor one-hot encoded.
Primero crea un tensor zero del tamaño 10 (el número de etiquetas en nuestro conjunto de datos)
y llama a [scatter](https://pytorch.org/docs/stable/generated/torch.Tensor.scatter_.html) el cual asigna un
``value=1`` en el índice dado por la etiqueta ``y``.

In [ ]:
target_transform = Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))

In [ ]:
print(torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(8), value=1))

---

## Lectura Adicional

- [Transformaciones de TorchVision](https://pytorch.org/vision/stable/transforms.html)

# Construir la Red Neuronal

Las redes neuronales se componen de capas/módulos que realizan operaciones en datos.
El espacio de nombres [torch.nn](https://pytorch.org/docs/stable/nn.html) proporciona todos los bloques de construcción que necesitas para
construir tu propia red neuronal. Cada módulo en PyTorch es una subclase de [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
Una red neuronal es un módulo en sí mismo que consiste en otros módulos (capas). Esta estructura anidada permite
construir y gestionar arquitecturas complejas fácilmente.

En las siguientes secciones, construiremos una red neuronal para clasificar imágenes en el conjunto de datos FashionMNIST.

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Obtener Dispositivo para Entrenamiento

Queremos poder entrenar nuestro modelo en un [acelerador](https://pytorch.org/docs/stable/torch.html#accelerators)
como CUDA, MPS, MTIA, o XPU. Si el acelerador actual está disponible, lo usaremos. De lo contrario, usamos la CPU.

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Usando dispositivo {device}")

## Definir la Clase

Definimos nuestra red neuronal heredando de ``nn.Module``, e
inicializamos las capas de la red neuronal en ``__init__``. Cada subclase de ``nn.Module`` implementa
las operaciones en datos de entrada en el método ``forward``.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Creamos una instancia de ``NeuralNetwork``, la movemos al ``device``, e imprimimos
su estructura.

In [ ]:
model = NeuralNetwork().to(device)
print(model)

Para usar el modelo, le pasamos los datos de entrada. Esto ejecuta el ``forward`` del modelo,
junto con algunas [operaciones en segundo plano](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
¡No llames a ``model.forward()`` directamente!

Llamar al modelo con la entrada devuelve un tensor bidimensional con dim=0 correspondiente a cada salida de 10 valores predichos sin procesar para cada clase, y dim=1 correspondiente a los valores individuales de cada salida.
Obtenemos las probabilidades de predicción pasándolo a través de una instancia del módulo ``nn.Softmax``.

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

---

## Capas del Modelo

Desglosemos las capas en el modelo FashionMNIST. Para ilustrarlo,
tomaremos un minilote de muestra de 3 imágenes de tamaño 28x28 y veremos qué le sucede mientras
lo pasamos a través de la red.

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

### nn.Flatten

Inicializamos la capa [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
para convertir cada imagen 2D de 28x28 en un arreglo contiguo de 784 valores de píxeles (
la dimensión del minilote (en dim=0) se mantiene).

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

### nn.Linear

La [capa lineal](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
es un módulo que aplica una transformación lineal en la entrada usando sus pesos y sesgos almacenados.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

### nn.ReLU

Las activaciones no lineales son las que crean las asignaciones complejas entre las entradas y salidas del modelo.
Se aplican después de las transformaciones lineales para introducir *no linealidad*, ayudando a las redes neuronales
a aprender una amplia variedad de fenómenos.

En este modelo, usamos [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) entre nuestras
capas lineales, pero hay otras activaciones para introducir no linealidad en tu modelo.

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) es un contenedor ordenado
de módulos. Los datos se pasan a través de todos los módulos en el mismo orden en que se definen. Puedes usar
contenedores secuenciales para armar rápidamente una red como ``seq_modules``.

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits

### nn.Softmax

La última capa lineal de la red neuronal devuelve `logits` - valores sin procesar en [-∞, ∞] - que se pasan al
módulo [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html). Los logits se escalan a valores
[0, 1] que representan las probabilidades predichas del modelo para cada clase. El parámetro ``dim`` indica la dimensión a lo largo de
la cual los valores deben sumar 1.

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab*100

## Parámetros del Modelo

Muchas capas dentro de una red neuronal están *parametrizadas*, es decir, tienen pesos
y sesgos asociados que se optimizan durante el entrenamiento. Heredar de ``nn.Module`` automáticamente
rastrea todos los campos definidos dentro de tu objeto modelo, y hace que todos los parámetros
sean accesibles usando los métodos ``parameters()`` o ``named_parameters()`` de tu modelo.

En este ejemplo, iteramos sobre cada parámetro, e imprimimos su tamaño y una vista previa de sus valores.

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

---

## Lectura Adicional

- [torch.nn API](https://pytorch.org/docs/stable/nn.html)

# Optimizando los Parámetros del Modelo

Ahora que tenemos un modelo y datos, es tiempo de entrenar, validar y probar nuestro modelo optimizando sus parámetros en nuestros datos. Entrenar un modelo es un proceso iterativo; en cada iteración el modelo hace una suposición sobre la salida, calcula el error en su suposición (*pérdida*), recolecta las derivadas del error con respecto a sus parámetros (como vimos en la sección anterior), y **optimiza** estos parámetros usando descenso de gradiente. Para un recorrido más detallado de este proceso, revisa este video en [backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

## Código Prerrequisito
Cargamos el código de las secciones anteriores en `Datasets & DataLoaders` y `Build Model`.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Obtener datos de entrenamiento y prueba
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hiperparámetros

Los hiperparámetros son parámetros ajustables que te permiten controlar el proceso de optimización del modelo. Diferentes valores de hiperparámetros pueden impactar el entrenamiento del modelo y las tasas de convergencia ([leer más sobre ajuste de hiperparámetros](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html))

Definimos los siguientes hiperparámetros para el entrenamiento:
 - **Número de Épocas** - el número de veces para iterar sobre el conjunto de datos
 - **Tamaño de Lote** - el número de muestras de datos propagadas a través de la red antes de que los parámetros se actualicen
 - **Tasa de Aprendizaje** - cuánto actualizar los parámetros del modelo en cada lote/época. Valores más pequeños resultan en una velocidad de aprendizaje lenta, mientras que valores grandes pueden resultar en un comportamiento impredecible durante el entrenamiento.

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Bucle de Optimización

Una vez que establecemos nuestros hiperparámetros, podemos entrenar y optimizar nuestro modelo con un bucle de optimización. Cada iteración del bucle de optimización se llama una **época**. 

Cada época consiste de dos partes principales:
 - **El Bucle de Entrenamiento** - itera sobre el conjunto de datos de entrenamiento e intenta converger a parámetros óptimos.
 - **El Bucle de Validación/Prueba** - itera sobre el conjunto de datos de prueba para verificar si el rendimiento del modelo está mejorando.

Vamos a familiarizarnos brevemente con algunos de los conceptos usados en el bucle de entrenamiento. Adelante, salta para ver la [Implementación Completa](#implementación-completa) del bucle de optimización.

### Función de Pérdida

Cuando se presenta con algunos datos de entrenamiento, nuestro modelo no entrenado probablemente no dará la respuesta correcta. **Función de pérdida** mide el grado de disimilitud del resultado obtenido al valor objetivo, y es la función de pérdida lo que queremos minimizar durante el entrenamiento. Para calcular la pérdida sumamos las predicciones de nuestro modelo en los puntos de datos dados y las comparamos contra los valores objetivo reales.

Las funciones de pérdida comunes incluyen [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (Error Cuadrático Medio) para tareas de regresión, y [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (Pérdida Logarítmica Negativa) para clasificación. [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) combina ``nn.LogSoftmax`` y ``nn.NLLLoss``.

Pasamos las predicciones de salida de nuestro modelo a ``nn.CrossEntropyLoss``, que normalizará las predicciones y calculará la pérdida.

In [ ]:
# Inicializar la función de pérdida
loss_fn = nn.CrossEntropyLoss()

### Optimizador

La optimización es el proceso de ajustar los parámetros del modelo para reducir el error del modelo en cada paso de entrenamiento. **Algoritmos de optimización** definen cómo se realiza este proceso (en este ejemplo usamos Descenso de Gradiente Estocástico). Toda la lógica de optimización está encapsulada en el objeto ``optimizer``. Aquí, usamos el optimizador SGD; además, hay muchos [diferentes optimizadores](https://pytorch.org/docs/stable/optim.html) disponibles en PyTorch tales como ADAM y RMSProp, que funcionan mejor para diferentes tipos de modelos y datos.

Inicializamos el optimizador registrando los parámetros del modelo que necesitan ser entrenados, y pasando el hiperparámetro de la tasa de aprendizaje.

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Dentro del bucle de entrenamiento, la optimización sucede en tres pasos:
 * Llama a ``optimizer.zero_grad()`` para reiniciar los gradientes de los parámetros del modelo. Los gradientes por defecto se suman; para prevenir el doble conteo, los ponemos explícitamente en cero en cada iteración.
 * Retropropaga la pérdida de predicción con una llamada a ``loss.backward()``. PyTorch deposita los gradientes de la pérdida con respecto a cada parámetro.
 * Una vez que tenemos nuestros gradientes, llamamos a ``optimizer.step()`` para ajustar los parámetros por los gradientes recolectados en el paso hacia atrás.

## Implementación Completa
Definimos las funciones ``train_loop`` que recorre el bucle de optimización, y ``test_loop`` que evalúa el rendimiento del modelo contra nuestros datos de prueba.

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Calcular predicción y pérdida
        pred = model(X)
        loss = loss_fn(pred, y)

        # Retropropagación
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"pérdida: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Rendimiento de Prueba: \n Precisión: {(100*correct):>0.1f}%, Pérdida Promedio: {test_loss:>8f} \n")

Inicializamos la función de pérdida y optimizador, y los pasamos a ``train_loop`` y ``test_loop``.
Siéntete libre de incrementar el número de épocas para rastrear el rendimiento de mejora del modelo.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Época {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("¡Terminado!")

## Inferencia del modelo

In [ ]:
classes = [
    "Camiseta/top",
    "Pantalón",
    "Suéter",
    "Vestido",
    "Abrigo",
    "Sandalia",
    "Camisa",
    "Zapatilla",
    "Bolsa",
    "Bota",
]

# Mostrar las primeras 4 imágenes de los conjuntos de datos de entrenamiento y prueba
import matplotlib.pyplot as plt


# Crear subgráficas para los datos de entrenamiento
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
fig.suptitle('Conjunto de Datos Fashion-MNIST - Primeras 4 Imágenes', fontsize=16)

# Mostrar las primeras 4 imágenes de entrenamiento
for i in range(4):
    image, label = training_data[i]
    axes[0, i].imshow(image.squeeze(), cmap='gray')
    axes[0, i].set_title(f'Entrena: {classes[label]}')
    axes[0, i].axis('off')

# Mostrar las primeras 4 imágenes de prueba
for i in range(4):
    image, label = test_data[i]
    axes[1, i].imshow(image.squeeze(), cmap='gray')
    axes[1, i].set_title(f'Prueba: {classes[label]}')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

# Imprimir detalles de las imágenes
print(f"Tamaño del conjunto de entrenamiento: {len(training_data)}")
print(f"Tamaño del conjunto de prueba: {len(test_data)}")
print(f"Forma de la imagen: {training_data[0][0].shape}")
print(f"Número de clases: {len(classes)}")

---

## Lectura Adicional
- [Función de Pérdida](https://pytorch.org/docs/stable/nn.html#loss-functions)
- [torch.optim](https://pytorch.org/docs/stable/optim.html)
- [Tutorial de Warmstart Training](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)

## 7. Guardar Modelo

PyTorch proporciona diferentes formas de guardar modelos. Puedes guardar solo los pesos del modelo o también el estado completo incluyendo el optimizador, la época actual, y otras métricas importantes para poder reanudar el entrenamiento.

### 7.1 Guardar solo los pesos del modelo

La forma más simple es guardar únicamente los parámetros (pesos) del modelo usando `state_dict()`. Esto es útil cuando solo necesitas el modelo entrenado para hacer inferencia.

In [ ]:
import os

# Crear directorio para guardar modelos si no existe
os.makedirs('modelos_guardados', exist_ok=True)

# Guardar solo los pesos del modelo
torch.save(model.state_dict(), 'modelos_guardados/modelo_pesos.pth')
print("✓ Modelo guardado en 'modelos_guardados/modelo_pesos.pth'")

# Para cargar el modelo más tarde:
# model = NeuralNetwork()  # Primero crear la arquitectura
# model.load_state_dict(torch.load('modelos_guardados/modelo_pesos.pth'))
# model.eval()  # Poner en modo evaluación

### 7.2 Guardar checkpoint completo

Un checkpoint completo incluye no solo los pesos del modelo, sino también el estado del optimizador, la época actual, el loss, y cualquier otra información necesaria para reanudar el entrenamiento exactamente donde lo dejaste.

In [ ]:
# Guardar un checkpoint completo
epoch = 5
loss_value = 0.25

checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss_value,
}

torch.save(checkpoint, 'checkpoint_epoch_5.pth')
print(f"✓ Checkpoint guardado en época {epoch}")

# Para cargar el checkpoint y reanudar el entrenamiento:
# checkpoint = torch.load('modelos_guardados/checkpoint_epoch_5.pth')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

### 7.3 Guardar checkpoints durante el entrenamiento

Ejemplo práctico: guardar un checkpoint cada N épocas durante el entrenamiento. Esto es muy útil para entrenamientos largos donde puedes querer guardar el progreso periódicamente.

In [ ]:
def train_with_checkpoints(model, train_dataloader, optimizer, loss_fn, epochs, device, save_every=5):
    """
    Función de entrenamiento que guarda checkpoints cada N épocas
    
    Args:
        model: El modelo a entrenar
        train_dataloader: DataLoader con datos de entrenamiento
        optimizer: Optimizador
        loss_fn: Función de pérdida
        epochs: Número total de épocas
        device: Dispositivo (CPU o GPU)
        save_every: Guardar checkpoint cada N épocas
    """
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        
        for batch, (X, y) in enumerate(train_dataloader):
            X, y = X.to(device), y.to(device)
            
            # Forward pass
            pred = model(X)
            loss = loss_fn(pred, y)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_dataloader)
        print(f"Época {epoch+1}/{epochs} - Loss promedio: {avg_loss:.4f}")
        
        # Guardar checkpoint cada 'save_every' épocas
        if (epoch + 1) % save_every == 0:
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
            }
            checkpoint_path = f'modelos_guardados/checkpoint_epoch_{epoch+1}.pth'
            torch.save(checkpoint, checkpoint_path)
            print(f"  ✓ Checkpoint guardado: {checkpoint_path}")
    
    print("\n✓ Entrenamiento completado!")

Ejemplo de uso: entrenar el modelo guardando checkpoints cada 2 épocas

In [ ]:
# Ejemplo: Entrenar guardando checkpoints cada 2 épocas
# Nota: Este es un ejemplo, asegúrate de tener train_dataloader definido

train_with_checkpoints(
    model=model,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=10,
    device=device,
    save_every=2  # Guardar cada 2 épocas
)

### 7.4 Cargar un checkpoint para reanudar entrenamiento

Si el entrenamiento se interrumpe, puedes cargar el último checkpoint y continuar desde donde lo dejaste.

In [ ]:
# Cargar un checkpoint guardado
def load_checkpoint(model, optimizer, checkpoint_path):
    """
    Carga un checkpoint y restaura el estado del modelo y optimizador
    
    Returns:
        epoch: La época desde donde se continúa
        loss: El loss del checkpoint
    """
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    print(f"✓ Checkpoint cargado desde época {epoch}")
    print(f"  Loss anterior: {loss:.4f}")
    
    return epoch, loss

# Ejemplo de uso:
start_epoch, last_loss = load_checkpoint(model, optimizer, 'modelos_guardados/checkpoint_epoch_6.pth')
# Ahora puedes continuar el entrenamiento desde start_epoch + 1

### 7.5 Mejores prácticas

**Consejos para guardar modelos:**

1. **Guardar el mejor modelo**: Además de guardar checkpoints periódicos, guarda el modelo con el mejor desempeño (menor loss de validación o mejor accuracy)

2. **Nombrar archivos claramente**: Usa nombres descriptivos como `modelo_epoch_10_loss_0.25.pth`

3. **Limpiar checkpoints antiguos**: Para ahorrar espacio, puedes mantener solo los últimos N checkpoints

4. **Guardar en diferentes formatos**: 
   - `.pth` o `.pt`: formato estándar de PyTorch
   - También puedes guardar metadatos adicionales como hiperparámetros del modelo

In [ ]:
# Ejemplo: Guardar el mejor modelo durante el entrenamiento
best_loss = float('inf')
best_model_path = 'modelos_guardados/mejor_modelo.pth'

# Durante el entrenamiento (dentro del loop de épocas):
# if current_loss < best_loss:
#     best_loss = current_loss
#     torch.save({
#         'epoch': epoch,
#         'model_state_dict': model.state_dict(),
#         'optimizer_state_dict': optimizer.state_dict(),
#         'loss': best_loss,
#     }, best_model_path)
#     print(f"  ✓ Nuevo mejor modelo guardado! Loss: {best_loss:.4f}")